<a href="https://colab.research.google.com/github/chugcha/hw_proga/blob/main/hw_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание 2**

# Загрузка книги и всего остального

In [195]:
!pip install pymystem3 --q
!pip install nltk --q
!pip install pymorphy3 --q
!pip install spacy --q
!pip install typing_extensions --upgrade --q
!python -m spacy download ru_core_news_sm --q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 44.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [196]:
!wget https://raw.githubusercontent.com/chugcha/hw_proga/refs/heads/main/bazhov1tom.txt
# Это файл из моего репозитория на гитхабе

with open('bazhov1tom.txt', 'r') as file:
   book_not_lower = file.read()
   book = book_not_lower.lower()

--2025-10-24 20:59:14--  https://raw.githubusercontent.com/chugcha/hw_proga/refs/heads/main/bazhov1tom.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 917244 (896K) [text/plain]
Saving to: ‘bazhov1tom.txt.1’

bazhov1tom.txt.1    100%[===================>] 895.75K  --.-KB/s    in 0.05s   

2025-10-24 20:59:14 (18.3 MB/s) - ‘bazhov1tom.txt.1’ saved [917244/917244]



# NLTK + Pymorphy

In [197]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt_tab')

book_tokens = word_tokenize(book, language='russian')
sw = stopwords.words('russian')

dirty_words = [w.lower() for w in book_tokens if w.isalpha()]
clean_words = [w for w in dirty_words if w not in sw]
# теперь в clean_words остались только все не-стоп-слова

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [198]:
import json
from pymorphy3 import MorphAnalyzer
from pprint import pprint

morph = MorphAnalyzer()

ana_w = [morph.parse(word) for word in clean_words]
# здесь записаны все возможные разборы слов из текста

with open('analysed_tokens_pymorphy.jsonl', 'w', encoding='utf-8') as f:
    for i in ana_w:
        token_data = {"word": i[0].word, "lemma": i[0].normalized.word, "pos": i[0].tag.POS}
        f.write(json.dumps(token_data) + '\n')

# теперь всё есть в json файле, смотрите:

with open('analysed_tokens_pymorphy.jsonl', 'r', encoding='utf-8') as k:
    for i in range(6):
        print(json.loads(k.readline()))

{'word': 'бажов', 'lemma': 'баж', 'pos': 'NOUN'}
{'word': 'павел', 'lemma': 'павел', 'pos': 'NOUN'}
{'word': 'собрание', 'lemma': 'собрание', 'pos': 'NOUN'}
{'word': 'сочинений', 'lemma': 'сочинение', 'pos': 'NOUN'}
{'word': 'трёх', 'lemma': 'три', 'pos': 'NUMR'}
{'word': 'томах', 'lemma': 'том', 'pos': 'NOUN'}


In [ ]:
#заносим информацию по каждой лемме в словарик для конвертирования в датафрейм
data_dict = {'lemmas': [], 'pos': [], 'frequency': []}

for i in ana_w:
  if i[0].normalized.word not in data_dict['lemmas']: # добавляем уникальную лемму
      data_dict['lemmas'].append(i[0].normalized.word)
      data_dict['pos'].append(i[0].normalized.tag.POS)
      data_dict['frequency'].append(1)

  else:
      # оказалось, бывает, что одну лемму определяют как разные части речи
      if data_dict['pos'][data_dict['lemmas'].index(i[0].normalized.word)] != i[0].normalized.tag.POS:
          same_lemmas_POS = []
          same_lemmas = [j for j, item in enumerate(data_dict['lemmas']) if item == i[0].normalized.word]
          for m in same_lemmas:
              same_lemmas_POS.append(data_dict['pos'][m])
          if i[0].normalized.tag.POS not in same_lemmas_POS:
              data_dict['lemmas'].append(i[0].normalized.word)
              data_dict['pos'].append(i[0].normalized.tag.POS)
              data_dict['frequency'].append(1)
      else:
          data_dict['frequency'][data_dict['lemmas'].index(i[0].normalized.word)] += 1
# это на всякий случай, проверить, что работает
print('Всего лексем:', len(data_dict['lemmas']))
print('Абсолютная частота вхождений лексемы "кошка":', data_dict['frequency'][data_dict['lemmas'].index('кошка')])


In [ ]:
import pandas as pd
bazhov_df = pd.DataFrame(data_dict)
print(bazhov_df.info())
print('\n')
#видно, что у некоторых слов не хватает каких-то значений POS. Выведем их
words_without_pos = bazhov_df[bazhov_df['pos'].isna()]
print(f"Слова без части речи: {len(words_without_pos)}")
print(words_without_pos['lemmas'].to_list())


In [ ]:
sorted_df = bazhov_df.sort_values('frequency', ascending=False)
print(sorted_df)

In [ ]:
import csv

sorted_df.to_csv('pymorphy_frequency.csv')

# Spacy

In [ ]:
import spacy
from spacy.lang.ru.examples import sentences


nlp = spacy.load('ru_core_news_sm')

In [ ]:
# очищаем текст от всего ненужного

cleaned_book = []
doc_sad = nlp(book)
for token in doc_sad:
  if not token.is_stop and not token.is_punct and not token.is_space and not token.is_digit:
      cleaned_book.append(token.text)
doc_happy = nlp(' '.join(clean_words))

In [ ]:
print(cleaned_book[:10])

In [ ]:
# так же, как было с pymorphy и nltk
with open('analysed_tokens_spacy.jsonl', 'w', encoding='utf-8') as f:
    for token in doc_happy:
        token_data = {"word": token.text, "lemma": token.lemma_, "pos": token.pos_}
        f.write(json.dumps(token_data) + '\n')

# вроде бы всё на месте
with open('analysed_tokens_spacy.jsonl', 'r', encoding='utf-8') as k:
    for i in range(6):
        print(json.loads(k.readline()))

In [ ]:
# делаем всё аналогично с pymorph
data_dict_sp = {'lemmas': [], 'pos': [], 'frequency': []}

for token in doc_happy:
  if token.lemma_ not in data_dict_sp['lemmas']: # добавляем уникальную лемму
      data_dict_sp['lemmas'].append(token.lemma_)
      data_dict_sp['pos'].append(token.pos_)
      data_dict_sp['frequency'].append(1)

  else:
      # оказалось, бывает, что одну лемму определяют как разные части речи
      if data_dict_sp['pos'][data_dict_sp['lemmas'].index(token.lemma_)] != token.pos_:
          same_lemmas_POS = []
          same_lemmas = [j for j, item in enumerate(data_dict_sp['lemmas']) if item == token.lemma_]
          for m in same_lemmas:
              same_lemmas_POS.append(data_dict_sp['pos'][m])
          if token.pos_ not in same_lemmas_POS:
              data_dict_sp['lemmas'].append(token.lemma_)
              data_dict_sp['pos'].append(token.pos_)
              data_dict_sp['frequency'].append(1)
      else:
          data_dict_sp['frequency'][data_dict_sp['lemmas'].index(token.lemma_)] += 1
# это на всякий случай, проверить, что работает
print('Всего лексем:', len(data_dict_sp['lemmas']))
print('Абсолютная частота вхождений лексемы "кошка":', data_dict_sp['frequency'][data_dict_sp['lemmas'].index('кошка')])


In [ ]:
bazhov_sp = pd.DataFrame(data_dict_sp)
print(bazhov_sp.info(), '\n')

sorted_df_sp = bazhov_sp.sort_values('frequency', ascending=False)
print('А вот и частотность: \n')
print(sorted_df_sp)

sorted_df_sp.to_csv('spacy_frequency.csv')

# Анализ частотности морфем

In [ ]:
print('Количество лексем из анализа pymorphy: ', bazhov_df.shape[0])
print('Количество лексем из анализа spacy: ', bazhov_sp.shape[0], '\n')

print('Набор морфем pymorph: ', bazhov_df["pos"].unique(), '\n')
print('Набор морфем spacy: ', bazhov_sp["pos"].unique())

In [ ]:
# получились некрасивые наборы, приведём их к общему виду:
def rename_pos_pymorphy(pos):
    if pos is None:
      return 'X'
    elif pos == 'NUMR':
        return 'NUM'
    elif pos == 'ADJF' or pos == 'ADJS':
        return 'ADJ'
    elif pos == 'NPRO':
        return 'PRON'
    elif pos == 'INFN':
        return 'VERB'
    elif pos == 'ADVB':
        return 'ADV'
    elif pos == 'PRCL':
        return 'PART'
    else:
        return pos

def rename_pos_spacy(pos):
    if pos == 'CCONJ' or pos == 'SCONJ':
        return 'CONJ'
    elif pos == 'PROPN':
        return 'NOUN'
    elif pos == 'ADP':
        return 'PREP'
    elif pos == 'AUX':
        return 'PRED'
    elif pos == 'DET':
        return 'ADJ'
    else:
        return pos

bazhov_df["pos"] = bazhov_df["pos"].apply(rename_pos_pymorphy)
bazhov_sp["pos"] = bazhov_sp["pos"].apply(rename_pos_spacy)

print('Новый набор морфем pymorph: ', bazhov_df["pos"].unique(), '\n')
print('Новый набор морфем spacy: ', bazhov_sp["pos"].unique())

# Получилось, что у нас в pymprph также выделены междометия,
# которые в spacy были удалены как стоп-слова. Но я не хочу исключать их из анализа и оставлю.

In [ ]:
def count_frec_pm(whatever):
   return (int(whatever) / len(clean_words))

def count_frec_sp(whatever):
   return (int(whatever) / len(cleaned_book))

morph_freq_pm = bazhov_df.groupby('pos')['frequency'].sum()
morph_freq_sp = bazhov_sp.groupby('pos')['frequency'].sum()
morph_freq_pm = morph_freq_pm.apply(count_frec_pm).sort_values(ascending=False)
morph_freq_sp = morph_freq_sp.apply(count_frec_sp).sort_values(ascending=False)
morph_freq_pm = morph_freq_pm.round(5)
morph_freq_sp = morph_freq_sp.round(5)

print('Доля частей речи по pymorph:')
print(morph_freq_pm, '\n')
print('Доля частей речи по spacy:')
print(morph_freq_sp)

По этим данным можно сделать несколько выводов. Во-первых, количество выделенных сушествительных, глаголов, прилагательных и наречий примерно равно, наибольшее сходство есть в частотности глаголов. Та разница, которая видна в доле существительных, прилагательных и местоимений, вероятно, возникла из-за того, что pymorph оказался менее чувствительным к формам, которые spacy смог определить, либо из-за того, что spacy определил их как существительные, что объясняет их большую долю в анализе spacy. То, что spacy выделяет большее количество местоимений, объясняется особенностью фильтрации стоп-слов, которые он применял.

# Визуализация 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [ ]:
X_pm = morph_freq_pm.index.tolist()
Y_pm = morph_freq_pm.values.tolist()

X_sp = morph_freq_sp.index.tolist()
Y_sp = morph_freq_sp.values.tolist()

print('Pymorphy: \n', X_pm, '\n', Y_pm, '\n')
print('Spacy: \n', X_sp, '\n', Y_sp)

In [ ]:
plt.figure(figsize=(15, 5), dpi=300)\

plt.subplot(1, 2, 1)
plt.title('По анализу Pymorph')
plt.ylabel('Относительная частотность')
plt.xlabel('Часть речи')
plt.bar(X_pm, Y_pm, color='hotpink')
plt.xticks(ticks=X_pm, labels=X_pm)
plt.ylim(0, 0.45)

plt.subplot(1, 2, 2)
plt.title('По анализу Spacy')
plt.xlabel('Часть речи')
plt.bar(X_sp, Y_sp, color='gold')
plt.xticks(ticks=X_sp, labels=X_sp)
plt.ylim(0, 0.45)

plt.show()

# Анализ самых частотных частей речи

In [ ]:
frequent_verbs_pm = bazhov_df[bazhov_df['pos'].isin(['VERB', 'INFN'])].groupby('lemmas')['frequency'].sum()
top20_verbs_pm = frequent_verbs_pm.sort_values(ascending=False).head(20)

frequent_adv_pm = bazhov_df[bazhov_df['pos'] == 'ADVB'].groupby('lemmas')['frequency'].sum()
top20_adv_pm = frequent_adv_pm.sort_values(ascending=False).head(20)

verbs_sp = bazhov_sp[bazhov_sp['pos'] == 'VERB'].groupby('lemmas')['frequency'].sum()
top20_verbs_sp = verbs_sp.sort_values(ascending=False).head(20)

adv_sp = bazhov_sp[bazhov_sp['pos'] == 'ADV'].groupby('lemmas')['frequency'].sum()
top20_adv_sp = adv_sp.sort_values(ascending=False).head(20)

In [ ]:
top20_verbs_pm.head()

In [ ]:
top20_verbs_sp.head()

In [ ]:
top20_adv_sp.head()

In [ ]:
top20_adv_pm.head()